# Tennis match prediction -- Feature Engineering

## Goal
1. Create some feature interaction between existing variables
2. Normalization of variables
3. Feature selection
    * Statistical-based feature selection
    * Model-based feature selection
4. Pricipal component analysis

### Importing useful packages

In [77]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# set the dataframe displaying format
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

# supress warnings
import warnings
warnings.filterwarnings("ignore")

sns.set()
sns.set_style("whitegrid")

### Load data
We will first import the data we prepare from previous notebooks `training` and `training_no_pts`, it consists all the stats of previous 10 mathces that player 1 and player 2 of the current match we want to predict have played.

In [78]:
training_raw = pd.read_csv('../Data/training.csv')
training_raw_no_pts = pd.read_csv('../Data/training_no_pts.csv')

In [79]:
training_raw.head()

,player_1_round_order_1,player_1_match_order_1,player_1_tourney_singles_draw_1,player_1_tourney_conditions_1,player_1_tourney_surface_1,player_1_sets_won_1,player_1_games_won_1,player_1_tiebreaks_won_1,player_1_ranking_1,player_1_ranking_points_1,player_1_aces_1,player_1_double_faults_1,player_1_first_serves_in_1,player_1_first_serves_total_1,player_1_first_serve_points_won_1,player_1_first_serve_points_total_1,player_1_second_serve_points_won_1,player_1_second_serve_points_total_1,player_1_break_points_saved_1,player_1_break_points_serve_total_1,player_1_service_points_won_1,player_1_service_points_total_1,player_1_first_serve_return_won_1,player_1_first_serve_return_total_1,player_1_second_serve_return_won_1,player_1_second_serve_return_total_1,player_1_break_points_converted_1,player_1_break_points_return_total_1,player_1_service_games_played_1,player_1_return_games_played_1,player_1_return_points_won_1,player_1_return_points_total_1,player_1_total_points_won_1,player_1_total_points_total_1,player_1_relative_sets_won_1,player_1_relative_games_won_1,player_1_relative_tiebreaks_won_1,player_1_relative_ranking_1,player_1_relative_ranking_points_1,player_1_relative_aces_1,player_1_relative_double_faults_1,player_1_relative_first_serves_in_1,player_1_relative_first_serves_total_1,player_1_relative_first_serve_points_won_1,player_1_relative_first_serve_points_total_1,player_1_relative_second_serve_points_won_1,player_1_relative_second_serve_points_total_1,player_1_relative_break_points_saved_1,player_1_relative_break_points_serve_total_1,player_1_relative_service_points_won_1,player_1_relative_service_points_total_1,player_1_relative_first_serve_return_won_1,player_1_relative_first_serve_return_total_1,player_1_relative_second_serve_return_won_1,player_1_relative_second_serve_return_total_1,player_1_relative_break_points_converted_1,player_1_relative_break_points_return_total_1,player_1_relative_service_games_played_1,player_1_relative_return_games_played_1,player_1_relative_return_points_won_1,player_1_relative_return_points_total_1,player_1_relative_total_points_won_1,player_1_relative_total_points_total_1,player_1_win_1,player_2_round_order_1,player_2_match_order_1,player_2_tourney_singles_draw_1,player_2_tourney_conditions_1,player_2_tourney_surface_1,player_2_sets_won_1,player_2_games_won_1,player_2_tiebreaks_won_1,player_2_ranking_1,player_2_ranking_points_1,player_2_aces_1,player_2_double_faults_1,player_2_first_serves_in_1,player_2_first_serves_total_1,player_2_first_serve_points_won_1,player_2_first_serve_points_total_1,player_2_second_serve_points_won_1,player_2_second_serve_points_total_1,player_2_break_points_saved_1,player_2_break_points_serve_total_1,player_2_service_points_won_1,player_2_service_points_total_1,player_2_first_serve_return_won_1,player_2_first_serve_return_total_1,player_2_second_serve_return_won_1,player_2_second_serve_return_total_1,player_2_break_points_converted_1,player_2_break_points_return_total_1,player_2_service_games_played_1,player_2_return_games_played_1,player_2_return_points_won_1,player_2_return_points_total_1,player_2_total_points_won_1,player_2_total_points_total_1,player_2_relative_sets_won_1,player_2_relative_games_won_1,player_2_relative_tiebreaks_won_1,player_2_relative_ranking_1,player_2_relative_ranking_points_1,player_2_relative_aces_1,player_2_relative_double_faults_1,player_2_relative_first_serves_in_1,player_2_relative_first_serves_total_1,player_2_relative_first_serve_points_won_1,player_2_relative_first_serve_points_total_1,player_2_relative_second_serve_points_won_1,player_2_relative_second_serve_points_total_1,player_2_relative_break_points_saved_1,player_2_relative_break_points_serve_total_1,player_2_relative_service_points_won_1,player_2_relative_service_points_total_1,player_2_relative_first_serve_return_won_1,player_2_relative_first_serve_return_total_1,player_2_relative_second_serve_return_won_1,player_2_relative_second_serve_return_total_1,player_2_relative_break_

In [80]:
training_raw_no_pts.head()

,player_1_round_order_1,player_1_match_order_1,player_1_tourney_singles_draw_1,player_1_tourney_conditions_1,player_1_tourney_surface_1,player_1_sets_won_1,player_1_games_won_1,player_1_tiebreaks_won_1,player_1_ranking_1,player_1_aces_1,player_1_double_faults_1,player_1_first_serves_in_1,player_1_first_serves_total_1,player_1_first_serve_points_won_1,player_1_first_serve_points_total_1,player_1_second_serve_points_won_1,player_1_second_serve_points_total_1,player_1_break_points_saved_1,player_1_break_points_serve_total_1,player_1_service_points_won_1,player_1_service_points_total_1,player_1_first_serve_return_won_1,player_1_first_serve_return_total_1,player_1_second_serve_return_won_1,player_1_second_serve_return_total_1,player_1_break_points_converted_1,player_1_break_points_return_total_1,player_1_service_games_played_1,player_1_return_games_played_1,player_1_return_points_won_1,player_1_return_points_total_1,player_1_total_points_won_1,player_1_total_points_total_1,player_1_relative_sets_won_1,player_1_relative_games_won_1,player_1_relative_tiebreaks_won_1,player_1_relative_ranking_1,player_1_relative_aces_1,player_1_relative_double_faults_1,player_1_relative_first_serves_in_1,player_1_relative_first_serves_total_1,player_1_relative_first_serve_points_won_1,player_1_relative_first_serve_points_total_1,player_1_relative_second_serve_points_won_1,player_1_relative_second_serve_points_total_1,player_1_relative_break_points_saved_1,player_1_relative_break_points_serve_total_1,player_1_relative_service_points_won_1,player_1_relative_service_points_total_1,player_1_relative_first_serve_return_won_1,player_1_relative_first_serve_return_total_1,player_1_relative_second_serve_return_won_1,player_1_relative_second_serve_return_total_1,player_1_relative_break_points_converted_1,player_1_relative_break_points_return_total_1,player_1_relative_service_games_played_1,player_1_relative_return_games_played_1,player_1_relative_return_points_won_1,player_1_relative_return_points_total_1,player_1_relative_total_points_won_1,player_1_relative_total_points_total_1,player_1_win_1,player_2_round_order_1,player_2_match_order_1,player_2_tourney_singles_draw_1,player_2_tourney_conditions_1,player_2_tourney_surface_1,player_2_sets_won_1,player_2_games_won_1,player_2_tiebreaks_won_1,player_2_ranking_1,player_2_aces_1,player_2_double_faults_1,player_2_first_serves_in_1,player_2_first_serves_total_1,player_2_first_serve_points_won_1,player_2_first_serve_points_total_1,player_2_second_serve_points_won_1,player_2_second_serve_points_total_1,player_2_break_points_saved_1,player_2_break_points_serve_total_1,player_2_service_points_won_1,player_2_service_points_total_1,player_2_first_serve_return_won_1,player_2_first_serve_return_total_1,player_2_second_serve_return_won_1,player_2_second_serve_return_total_1,player_2_break_points_converted_1,player_2_break_points_return_total_1,player_2_service_games_played_1,player_2_return_games_played_1,player_2_return_points_won_1,player_2_return_points_total_1,player_2_total_points_won_1,player_2_total_points_total_1,player_2_relative_sets_won_1,player_2_relative_games_won_1,player_2_relative_tiebreaks_won_1,player_2_relative_ranking_1,player_2_relative_aces_1,player_2_relative_double_faults_1,player_2_relative_first_serves_in_1,player_2_relative_first_serves_total_1,player_2_relative_first_serve_points_won_1,player_2_relative_first_serve_points_total_1,player_2_relative_second_serve_points_won_1,player_2_relative_second_serve_points_total_1,player_2_relative_break_points_saved_1,player_2_relative_break_points_serve_total_1,player_2_relative_service_points_won_1,player_2_relative_service_points_total_1,player_2_relative_first_serve_return_won_1,player_2_relative_first_serve_return_total_1,player_2_relative_second_serve_return_won_1,player_2_relative_second_serve_return_total_1,player_2_relative_break_points_converted_1,player_2_relative_break_points_return_total_1,player_2_relative_service_games_played_1,player_2_relativ

### Feature interaction
In this part, we will apply some tennis domain knowledge to create some new features, when we take a look of data above, we can see that most of the variables are point basis, the traditional tennis stats will have many rate basis variables, such as `first serve in %`, `break points saving %`, `second serve winning %`...etc. we will take a deep look of all the existing variables to generate the desired features.

We have come up with the following stats we would like to generate:
1. `first_serve_winning_percentage` = `first_serve_points_won` / `first_serves_in`
2. `second_serve_winning_percentage` = `second_serve_points_won` / `second_serves_points_total`
3. `first_serve_percentage` = `first_serves_in` / `first_serves_total`
4. `service_points_winning_percentage` = `service_points_won` / `service_points_total`
5. `first_serve_return_winning_percentage` = `first_serve_return_won` / `first_serve_return_total`
6. `second_serve_return_winning_percentage` = `second_serve_return_won` / `second_serve_return_total`
7. `return_points_winning_percentage` = `return_points_won` / `return_points_total`
8. `total_points_winning_percentage` = `total_points_won` / `total_points_total`

We will define a helper function to help with the feature interaction we described above.

In [81]:
def feature_interaction(df_raw, drop_original=False):
    df = df_raw.copy()
    
    # extract all the stats information
    stats = list(df.columns.str.extract(r'player_1_relative_(.+)_1$').dropna()[0])
    
    # add oppoent stats from the relative stats   
    for i in range(1,11):
        for j in stats:
            df[f'player_1_opponent_{j}_{i}'] = df[f'player_1_{j}_{i}'] - df[f'player_1_relative_{j}_{i}']

        for j in stats:    
            df[f'player_2_opponent_{j}_{i}'] = df[f'player_2_{j}_{i}'] - df[f'player_2_relative_{j}_{i}']
        
    # define another function to add feature interaction 
    # define feature interaction
    new_feature_dict = {'first_serve_in_percentage': ('first_serves_in', 'first_serves_total'),
                    'first_serve_winning_percentage': ('first_serve_points_won', 'first_serves_in'),
                    'second_serve_points_winning_percentage': ('second_serve_points_won', 'second_serve_points_total'),
                    'service_points_winning_percentage': ('service_points_won', 'service_points_total'),
                    'first_serve_return_winning_percentage': ('first_serve_return_won', 'first_serve_return_total'),
                    'second_serve_return_winning_percentage': ('second_serve_return_won', 'second_serve_return_total'),
                    'return_points_winning_percentage': ('return_points_won', 'return_points_total'),
                    'total_points_winning_percentage': ('total_points_won', 'total_points_total')}
    
    def add_percentage_data(row, i, j, player):
            # set the condition, only to perform feature interaction when the stats make sense
            if row[f'{player}_{new_feature_dict[j][1]}_{i}'] != 0 and \
                row[f'{player}_{new_feature_dict[j][0]}_{i}'] <= row[f'{player}_{new_feature_dict[j][1]}_{i}']:

                result = row[f'{player}_{new_feature_dict[j][0]}_{i}'] / \
                                            row[f'{player}_{new_feature_dict[j][1]}_{i}']

            else: 
                result = np.nan

            return  result
    
    for i in range(1,11):
        for player in ['player_1', 'player_1_opponent', 'player_2', 'player_2_opponent']:
            for j in new_feature_dict:
                df[f'{player}_{j}_{i}'] = df.apply(add_percentage_data, args=(i, j, player), axis=1)
    
    # add the relative stats of the feature interaction columns
    for i in range(1,11):
        for j in new_feature_dict:
            df[f'player_1_relative_{j}_{i}'] = df[f'player_1_{j}_{i}'] - df[f'player_1_opponent_{j}_{i}']
            df[f'player_2_relative_{j}_{i}'] = df[f'player_2_{j}_{i}'] - df[f'player_2_opponent_{j}_{i}']
    
    # reorder the column
    all_col = df.columns
    reorder_col = []
    for i in range(1, 11):
        new = list(df.columns[df.columns.str.contains(r'_{}$'.format(i))])
        reorder_col += new
    
    
    df = df[reorder_col + ['outcome']]
    
    return df

We will include all the features we generated from `feature_interaction` function as well as keep all the original columns first. This might take a while to run.

In [83]:
training_all_features = feature_interaction(training_raw)
training_all_features_no_pts = feature_interaction(training_raw_no_pts)

After we have run the feature interaction, let us check whether there are missing values exists in any columns, recall that the original data `train_raw` did not have any missing values, however, we implemented function `feature_interaction` to filter out some impossible values and stats that does not make sense at all, for example, `first_serve_winning_percentage` is calculated by `first_serve_points_won` divided by `first_serve_points_in`, it will not make sense that an ATP level player has `first_serve_points_in` equals zero, this literally means he did not make any first serve for the entire match, which could be possible for an amateur, but definitely not an ATP player. The funciton took care of those cases and put the missing values in the cell if it occurred.

In [84]:
# check missing values in each column, showing only the top ten
training_all_features.isnull().sum().sort_values(ascending=False).head(10)

player_1_opponent_first_serve_winning_percentage_10            273
player_1_first_serve_return_winning_percentage_10              273
player_1_opponent_total_points_winning_percentage_10           273
player_1_opponent_return_points_winning_percentage_10          273
player_1_opponent_second_serve_return_winning_percentage_10    273
player_1_opponent_first_serve_return_winning_percentage_10     273
player_1_opponent_service_points_winning_percentage_10         273
player_1_opponent_second_serve_points_winning_percentage_10    273
player_1_opponent_first_serve_in_percentage_10                 273
player_1_total_points_winning_percentage_10                    273
dtype: int64

In [85]:
training_all_features_no_pts.isnull().sum().sort_values(ascending=False).head(10)

player_1_relative_first_serve_winning_percentage_1    519
player_1_relative_first_serve_winning_percentage_8    488
player_1_relative_first_serve_winning_percentage_2    484
player_2_relative_first_serve_winning_percentage_1    476
player_1_relative_first_serve_winning_percentage_9    475
player_1_relative_first_serve_winning_percentage_7    474
player_1_relative_first_serve_winning_percentage_4    474
player_2_relative_first_serve_winning_percentage_3    470
player_2_relative_first_serve_winning_percentage_2    470
player_1_relative_first_serve_winning_percentage_3    469
dtype: int64

We will get rid of the examples where there are missing values exists.

In [86]:
# drop rows with missing values
training_all_features_clean = training_all_features.dropna()
training_all_features_no_pts_clean = training_all_features_no_pts.dropna()

In [87]:
print(training_all_features.shape)
print(training_all_features_clean.shape)

(15529, 2341)
(12804, 2341)


We can see that we lost around 2700 of rows, which is accetable since we still have around 13K training examples. We will also drop some original columns that we generated the new columns from since there will exists multicollinearity issue, we will drop any opponent stats since we already have relative stats.

In [88]:
print(training_all_features_no_pts.shape)
print(training_all_features_no_pts_clean.shape)

(10823, 2281)
(7497, 2281)


In [90]:
opponent_col_1 = [i for i in training_all_features_clean.columns if 'opponent' in i]
print(len(opponent_col_1))

740


In [91]:
opponent_col_2 = [i for i in training_all_features_no_pts_clean.columns if 'opponent' in i]
print(len(opponent_col_2))

720


In [92]:
training_all_features_clean_drop_oppo = training_all_features_clean.drop(opponent_col_1, axis=1)
training_all_features_no_pts_clean_drop_oppo = training_all_features_no_pts_clean.drop(opponent_col_2, axis=1)

### Normalization
Since our variables in the dataset have different scales, it is important for us to normalize each column to have them under the same scale, without the normalization steps, the machine learning model may take forever to come up with a good fitting, we will use the most common way, z-score standardization for our data, which basically calculates the mean and standard deviation of each variable and substract the mean and divide by the standard deviation of that particular variable, this will make each vairable of mean = 0 and standard deviation = 1, we will import the package from sklearn to help with normalization. Before doing so, we will need to split the training and testing set since we are not supposed to know the mean and variance of the test set. 

#### Drop categorical variables
We notice there are several categorical vairiables when we first prepare our training examples, there are actaully few variance in each variable and might not be helpful to our model, we decide to drop those at this step since they are not supposed to be normalized either.

In [93]:
# show the unique values in each categorical variables
training_all_features_clean.select_dtypes(['object']).nunique().sort_values(ascending=False).head(10)

player_2_tourney_surface_10    3
player_1_tourney_surface_7     3
player_1_tourney_surface_1     3
player_2_tourney_surface_1     3
player_1_tourney_surface_2     3
player_2_tourney_surface_2     3
player_1_tourney_surface_3     3
player_2_tourney_surface_3     3
player_1_tourney_surface_4     3
player_2_tourney_surface_4     3
dtype: int64

In [94]:
training_all_features_no_pts_clean.select_dtypes(['object']).nunique().sort_values(ascending=False).head(10)

player_2_tourney_surface_10    4
player_1_tourney_surface_7     4
player_1_tourney_surface_1     4
player_2_tourney_surface_1     4
player_1_tourney_surface_2     4
player_2_tourney_surface_2     4
player_1_tourney_surface_3     4
player_2_tourney_surface_3     4
player_1_tourney_surface_4     4
player_2_tourney_surface_4     4
dtype: int64

We can see that there are at most three unique values in our categorical variables, this is not providing too much information.

In [95]:
# drop the categorical variables
cate_col_1 = training_all_features_clean.select_dtypes(['object']).columns
cate_col_2 = training_all_features_no_pts_clean.select_dtypes(['object']).columns

training_all_features_clean.drop(cate_col_1, axis=1, inplace=True)
training_all_features_clean_drop_oppo.drop(cate_col_1, axis=1, inplace=True)
training_all_features_no_pts_clean.drop(cate_col_2, axis=1, inplace=True)
training_all_features_no_pts_clean_drop_oppo.drop(cate_col_2, axis=1, inplace=True)

#### Numerical variables with few unique values

Since we also have some numerical features that do not have many unique values in it, we will do some analysis of those variables before doing normalization.

In [96]:
unique1 = training_all_features_clean.nunique()
unique2 = training_all_features_clean_drop_oppo.nunique()
unique3 = training_all_features_no_pts_clean.nunique()
unique4 = training_all_features_no_pts_clean_drop_oppo.nunique()
unique4.sort_values().head(30)

player_1_relative_total_points_total_9     1
player_1_relative_total_points_total_5     1
player_1_relative_total_points_total_6     1
player_1_relative_total_points_total_1     1
player_1_relative_total_points_total_2     1
player_1_relative_total_points_total_8     1
player_1_relative_total_points_total_7     1
player_2_relative_total_points_total_7     1
outcome                                    2
player_1_win_9                             2
player_2_win_8                             2
player_2_relative_total_points_total_8     2
player_1_win_1                             2
player_2_win_3                             2
player_2_win_7                             2
player_1_win_7                             2
player_1_win_4                             2
player_1_win_3                             2
player_2_win_6                             2
player_2_win_4                             2
player_1_win_6                             2
player_2_relative_total_points_total_1     2
player_2_w

Features with only one value definitely will not help, we will drop those.

In [97]:
drop_col_1 = unique1[unique1 == 1].index
drop_col_2 = unique2[unique2 == 1].index

training_all_features_clean.drop(drop_col_1, 1, inplace=True)
training_all_features_clean_drop_oppo.drop(drop_col_2, 1, inplace=True)

drop_col_3 = unique3[unique3 == 1].index
drop_col_4 = unique4[unique4 == 1].index

training_all_features_no_pts_clean.drop(drop_col_3, 1, inplace=True)
training_all_features_no_pts_clean_drop_oppo.drop(drop_col_4, 1, inplace=True)

We also notice that there are two unique values in `relative_total_points_total` related columns in `training_all_features_no_pts_clean`, this does not look right, they should all have zero values. We will drop those as well.

In [98]:
relative_col = [i for i in training_all_features_no_pts_clean if 'relative_total_points_total' in i]

In [99]:
# drop the columns
training_all_features_no_pts_clean.drop(relative_col, 1, inplace=True)
training_all_features_no_pts_clean_drop_oppo.drop(relative_col, 1, inplace=True)

We will keep the binary variables such as `player_1_win_1` since this indicates whether a player has won the previous matches or not, which might be informative for our future model. This will not be standardized either.

In [100]:
winning_col = [i for i in training_all_features_clean.columns if '_win_' in i]
winning_col_2 = [i for i in training_all_features_no_pts_clean.columns if '_win_' in i]

#### Split data into training and testing sets

In [101]:
# import package for train and test set split
from sklearn.model_selection import train_test_split

tennis_all_feature_clean_X_train, tennis_all_feature_clean_X_test, \
tennis_all_feature_clean_y_train, tennis_all_feature_clean_y_test = \
train_test_split(training_all_features_clean.drop(['outcome'],1), training_all_features_clean['outcome'],
                 test_size=0.3, random_state=1009)

tennis_all_feature_clean_drop_oppo_X_train, tennis_all_feature_clean_drop_oppo_X_test, \
tennis_all_feature_clean_drop_oppo_y_train, tennis_all_feature_clean_drop_oppo_y_test = \
train_test_split(training_all_features_clean_drop_oppo.drop(['outcome'],1), 
                 training_all_features_clean_drop_oppo['outcome'], test_size=0.3, random_state=1009)

In [44]:
tennis_all_feature_no_pts_clean_X_train, tennis_all_feature_no_pts_clean_X_test, \
tennis_all_feature_no_pts_clean_y_train, tennis_all_feature_no_pts_clean_y_test = \
train_test_split(training_all_features_no_pts_clean.drop(['outcome'],1), training_all_features_no_pts_clean['outcome'],
                 test_size=0.3, random_state=1009)

tennis_all_feature_no_pts_clean_drop_oppo_X_train, tennis_all_feature_no_pts_clean_drop_oppo_X_test, \
tennis_all_feature_no_pts_clean_drop_oppo_y_train, tennis_all_feature_no_pts_clean_drop_oppo_y_test = \
train_test_split(training_all_features_no_pts_clean_drop_oppo.drop(['outcome'],1), 
                 training_all_features_no_pts_clean_drop_oppo['outcome'], test_size=0.3, random_state=1009)

In [245]:
# import packages from sklearn
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# exclude the winning columns since they are binary features that do not need to be standardized 
col1 = list(set(tennis_all_feature_clean_X_train.columns) - set(winning_col))
tennis_all_feature_clean_X_train[col1] = scaler.fit_transform(tennis_all_feature_clean_X_train[col1])
tennis_all_feature_clean_X_test[col1] = scaler.transform(tennis_all_feature_clean_X_test[col1])

col2 = list(set(tennis_all_feature_clean_drop_oppo_X_train.columns) - set(winning_col))
tennis_all_feature_clean_drop_oppo_X_train[col2] = scaler.fit_transform(tennis_all_feature_clean_drop_oppo_X_train[col2])
tennis_all_feature_clean_drop_oppo_X_test[col2] = scaler.transform(tennis_all_feature_clean_drop_oppo_X_test[col2])

In [46]:
col3 = list(set(tennis_all_feature_no_pts_clean_X_train.columns) - set(winning_col_2))

tennis_all_feature_no_pts_clean_X_train[col3] = scaler.fit_transform(tennis_all_feature_no_pts_clean_X_train[col3])
tennis_all_feature_no_pts_clean_X_test[col3] = scaler.transform(tennis_all_feature_no_pts_clean_X_test[col3])

col4 = list(set(tennis_all_feature_no_pts_clean_drop_oppo_X_train.columns) - set(winning_col_2))

tennis_all_feature_no_pts_clean_drop_oppo_X_train[col4] = scaler.fit_transform(tennis_all_feature_no_pts_clean_drop_oppo_X_train[col4])
tennis_all_feature_no_pts_clean_drop_oppo_X_test[col4] = scaler.transform(tennis_all_feature_no_pts_clean_drop_oppo_X_test[col4])

### Feature selection
We can see from above our data has around 1600 features even we have dropped many of it already, this might create a problem of noise or slow down our model training process, we will perform feature selection in this part to select only the subset of k features instead of using all features in our model. We will use two different approaches, one is statistical-based and the other is model-based.

#### Statistical-based
We will use hypothesis testing to select only the best features from a dataset, these tests rely more on formalized statistical methods and use the p-value ranking for feature selection. We will utilize two functions from sklearn package `f_classif` and `SelectKBest`, `f_classif` function will perform ANOVA test on each feature on its own and assign that feature a p-value, `SelectKBest` will rank the features by that p-value(the lower the better) and keep only the best k features. 

In [246]:
# import sklean package
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest

# perform p_value ranking on tennis_all_feature_clean_X_train first
k_best = SelectKBest(f_classif)
k_best.fit(tennis_all_feature_clean_X_train, tennis_all_feature_clean_y_train)

SelectKBest(k=10, score_func=<function f_classif at 0x1a18fb6620>)

In [247]:
p_value_rank_1 = pd.DataFrame({'column': tennis_all_feature_clean_X_train.columns, 'p_value': k_best.pvalues_}) \
.sort_values('p_value').set_index('column')

In [248]:
p_value_rank_1.head(15)

,p_value
column,
player_1_round_order_10,0.000000e+00
player_2_round_order_10,0.000000e+00
player_2_opponent_ranking_points_10,3.805463e-160
player_1_opponent_ranking_points_10,4.249147e-153
player_1_match_order_10,1.127549e-152
player_2_match_order_10,6.962604e-129
player_1_opponent_ranking_10,4.094031e-126
player_2_opponent_ranking_10,5.688461e-105
player_2_round_order_9,7.580916e-94


From the table above, we can see the top 15 features that has the lowest p-value, there is an interesting fact to point out, those variables are mostly from the stats of the previous match of the player, which is the "closest" match to the current match a player is going to play, this kind of make sense, as we mentioned that the closest match should be the match that best reflecting a player's condition. We will use the top 500 features.

In [249]:
# keep only top 500
keep_col_500_1 = p_value_rank_1.index[:500]

In [250]:
tennis_all_feature_clean_X_train_p_value = tennis_all_feature_clean_X_train[keep_col_500_1].copy()
tennis_all_feature_clean_X_test_p_value = tennis_all_feature_clean_X_test[keep_col_500_1].copy()

We will repeat the same process on `tennis_all_feature_clean_drop_oppo_X_train` and `tennis_all_feature_clean_drop_oppo_X_test`

In [251]:
k_best.fit(tennis_all_feature_clean_drop_oppo_X_train, tennis_all_feature_clean_drop_oppo_y_train)

SelectKBest(k=10, score_func=<function f_classif at 0x1a18fb6620>)

In [252]:
p_value_rank_2 = pd.DataFrame({'column': tennis_all_feature_clean_drop_oppo_X_train.columns, 'p_value': k_best.pvalues_}) \
.sort_values('p_value').set_index('column')

In [253]:
p_value_rank_2.head(15)

,p_value
column,
player_1_round_order_10,0.000000e+00
player_2_round_order_10,0.000000e+00
player_1_match_order_10,1.127549e-152
player_2_match_order_10,6.962604e-129
player_2_round_order_9,7.580916e-94
player_1_round_order_9,1.455200e-91
player_2_ranking_points_10,1.959878e-86
player_1_ranking_points_10,2.503592e-86
player_1_ranking_points_9,5.937269e-86


We can see that the best features are pretty similar and since `tennis_all_feature_clean_drop_oppo_X_train` does not have any opponent related stats, the best features almost overlapped with each other, we will also keep the top 500 in this case.

In [254]:
# keep only top 500
keep_col_500_2 = p_value_rank_2.index[:500]

In [255]:
tennis_all_feature_clean_drop_oppo_X_train_p_value = tennis_all_feature_clean_drop_oppo_X_train[keep_col_500_2].copy()
tennis_all_feature_clean_drop_oppo_X_test_p_value = tennis_all_feature_clean_drop_oppo_X_test[keep_col_500_2].copy()

In [50]:
k_best.fit(tennis_all_feature_no_pts_clean_X_train, tennis_all_feature_no_pts_clean_y_train)

SelectKBest(k=10, score_func=<function f_classif at 0x1a59c188c8>)

In [51]:
p_value_rank_3 = pd.DataFrame({'column': tennis_all_feature_no_pts_clean_X_train.columns, 'p_value': k_best.pvalues_}) \
.sort_values('p_value').set_index('column')

In [52]:
p_value_rank_3.head(15)

,p_value
column,
player_2_round_order_10,0.000000e+00
player_1_round_order_10,0.000000e+00
player_2_match_order_10,7.968676e-107
player_1_match_order_10,2.627700e-93
player_2_opponent_ranking_10,4.154399e-53
player_1_round_order_9,1.504552e-51
player_2_round_order_9,1.809806e-51
player_1_opponent_ranking_10,2.732591e-48
player_2_ranking_7,1.435851e-26


We can clearly see that the `ranking points` related columns in previous data were replaced by the `ranking number` related columns, since more or less they are providing the same information, however, we also notice that the p_values are a lot larger(though still small in absolute value) than `ranking points` related columns(around 1e-30 vs 1e-80). Which might suggest that `ranking points` provide a lot more information than `ranking number` itself. We will repeat for the last dataset `tennis_all_feature_no_pts_drop_oppo_clean_X_train`.

In [53]:
# keep only top 500
keep_col_500_3 = p_value_rank_3.index[:500]

In [54]:
tennis_all_feature_no_pts_clean_X_train_p_value =tennis_all_feature_no_pts_clean_X_train[keep_col_500_3].copy()
tennis_all_feature_no_pts_clean_X_test_p_value =tennis_all_feature_no_pts_clean_X_test[keep_col_500_3].copy()

In [55]:
k_best.fit(tennis_all_feature_no_pts_clean_drop_oppo_X_train, tennis_all_feature_no_pts_clean_drop_oppo_y_train)

SelectKBest(k=10, score_func=<function f_classif at 0x1a59c188c8>)

In [56]:
p_value_rank_4 = pd.DataFrame({'column': tennis_all_feature_no_pts_clean_drop_oppo_X_train.columns, 'p_value': k_best.pvalues_}) \
.sort_values('p_value').set_index('column')

In [57]:
p_value_rank_4.head(15)

,p_value
column,
player_2_round_order_10,0.000000e+00
player_1_round_order_10,0.000000e+00
player_2_match_order_10,7.968676e-107
player_1_match_order_10,2.627700e-93
player_1_round_order_9,1.504552e-51
player_2_round_order_9,1.809806e-51
player_2_ranking_7,1.435851e-26
player_2_ranking_6,1.886855e-23
player_2_ranking_10,1.768700e-22


Again, top rank features stayed the same, all `ranking number` related columns are still in top 15.

In [58]:
# keep only top 500
keep_col_500_4 = p_value_rank_4.index[:500]

In [59]:
tennis_all_feature_no_pts_clean_drop_oppo_X_train_p_value =tennis_all_feature_no_pts_clean_drop_oppo_X_train[keep_col_500_4].copy()
tennis_all_feature_no_pts_clean_drop_oppo_X_test_p_value =tennis_all_feature_no_pts_clean_drop_oppo_X_test[keep_col_500_4].copy()

#### Model-based
We will utilize tree-based model feature importance to select our features, tree starts at the root node and greedily chooses the optimal split at every junction that optimizes a certain metric of node purity. This can give us a rank of features base on the information gain.

In [60]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()

In [257]:
# first deal with tennis_all_feature_clean_X_train
dt.fit(tennis_all_feature_clean_X_train, tennis_all_feature_clean_y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [258]:
dt_feature_1 = pd.DataFrame({'column': tennis_all_feature_clean_X_train.columns, 'feature_importance': dt.feature_importances_}) \
.sort_values('feature_importance', ascending=False).set_index('column')

In [259]:
dt_feature_1.head(15)

,feature_importance
column,
player_2_round_order_10,0.383742
player_1_round_order_10,0.230862
player_1_tourney_singles_draw_10,0.050960
player_2_tourney_singles_draw_10,0.050180
player_2_ranking_8,0.007501
player_1_ranking_2,0.006019
player_1_ranking_8,0.005311
player_1_opponent_ranking_10,0.004544
player_2_sets_won_10,0.004514


It is interesting to see that statistical-based and model-based select pretty different features in terms of importance, however, we also notice that the top 2 features `player_2_round_order_10` and `player_1_round_order_10` matches the statistical-based selection. We will use the top 500 features again.

In [260]:
# keep only top 500
keep_col_500_5 = dt_feature_1.index[:500]

In [261]:
tennis_all_feature_clean_X_train_tree = tennis_all_feature_clean_X_train[keep_col_500_5].copy()
tennis_all_feature_clean_X_test_tree = tennis_all_feature_clean_X_test[keep_col_500_5].copy()

Again, we will repeat the process on `tennis_all_feature_clean_drop_oppo_X_train`.

In [262]:
# deal with tennis_all_feature_clean_X_drop_oppo_train
dt.fit(tennis_all_feature_clean_drop_oppo_X_train, tennis_all_feature_clean_drop_oppo_y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [263]:
dt_feature_2 = pd.DataFrame({'column': tennis_all_feature_clean_drop_oppo_X_train.columns, 'feature_importance': dt.feature_importances_}) \
.sort_values('feature_importance', ascending=False).set_index('column')

In [264]:
dt_feature_2.head(15)

,feature_importance
column,
player_2_round_order_10,0.383354
player_1_round_order_10,0.230862
player_1_tourney_singles_draw_10,0.052739
player_2_tourney_singles_draw_10,0.048638
player_2_ranking_8,0.007501
player_1_ranking_2,0.006019
player_1_ranking_8,0.005311
player_1_relative_ranking_points_10,0.004747
player_2_sets_won_10,0.004514


The result is similar to the first one except every opponent related stats are removed, we will use the top 500 features.

In [265]:
# keep only top 500
keep_col_500_6 = dt_feature_2.index[:500]

In [266]:
tennis_all_feature_clean_drop_oppo_X_train_tree = tennis_all_feature_clean_drop_oppo_X_train[keep_col_500_6].copy()
tennis_all_feature_clean_drop_oppo_X_test_tree = tennis_all_feature_clean_drop_oppo_X_test[keep_col_500_6].copy()

In [61]:
dt.fit(tennis_all_feature_no_pts_clean_X_train, tennis_all_feature_no_pts_clean_y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [62]:
dt_feature_3 = pd.DataFrame({'column': tennis_all_feature_no_pts_clean_X_train.columns, 'feature_importance': dt.feature_importances_}) \
.sort_values('feature_importance', ascending=False).set_index('column')

In [63]:
dt_feature_3.head(15)

,feature_importance
column,
player_2_round_order_10,0.430959
player_1_round_order_10,0.221913
player_1_tourney_singles_draw_10,0.055048
player_2_tourney_singles_draw_10,0.038016
player_1_opponent_second_serve_return_total_10,0.003956
player_2_second_serve_return_won_2,0.003850
player_1_opponent_first_serve_winning_percentage_4,0.003581
player_2_relative_ranking_10,0.003460
player_2_relative_second_serve_return_winning_percentage_9,0.003304


There are some interesting features showed up like `player_1_relative_first_serve_return_winning_percentage_7` and `player_2_aces_4` in top 15, which are more difficult to explain the relationship between the current playing outcome. This might be due to the noise that came out specifically in this dataset.

In [64]:
# keep only top 500
keep_col_500_7 = dt_feature_3.index[:500]

In [65]:
tennis_all_feature_no_pts_clean_X_train_tree =tennis_all_feature_no_pts_clean_X_train[keep_col_500_7].copy()
tennis_all_feature_no_pts_clean_X_test_tree =tennis_all_feature_no_pts_clean_X_test[keep_col_500_7].copy()

In [66]:
dt.fit(tennis_all_feature_no_pts_clean_drop_oppo_X_train, tennis_all_feature_no_pts_clean_drop_oppo_y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [67]:
dt_feature_4 = pd.DataFrame({'column': tennis_all_feature_no_pts_clean_drop_oppo_X_train.columns, \
        'feature_importance': dt.feature_importances_}).sort_values('feature_importance', ascending=False).set_index('column')

In [68]:
dt_feature_4.head(15)

,feature_importance
column,
player_2_round_order_10,0.433308
player_1_round_order_10,0.221913
player_1_tourney_singles_draw_10,0.055048
player_2_tourney_singles_draw_10,0.038016
player_1_second_serve_points_total_10,0.003956
player_2_second_serve_return_won_2,0.003850
player_2_relative_ranking_10,0.003840
player_2_ranking_8,0.003634
player_1_first_serve_return_winning_percentage_4,0.003581


In [69]:
# keep only top 500
keep_col_500_8 = dt_feature_4.index[:500]

In [70]:
tennis_all_feature_no_pts_clean_drop_oppo_X_train_tree =tennis_all_feature_no_pts_clean_drop_oppo_X_train[keep_col_500_8].copy()
tennis_all_feature_no_pts_clean_drop_oppo_X_test_tree =tennis_all_feature_no_pts_clean_drop_oppo_X_test[keep_col_500_8].copy()

### Principal component analysis (PCA)
We have completed the feature selection part and the next step we will perform another dimension reduction method called PCA, it is a method that converts the original correlated variables into a set of values that are linearly uncorrelated (orthogonal transformation), this method could retained the most of variances in each variables depends on how many components we include. The major advantage of PCA is the reduction of noise and get rid of the redundancy of data, we will use the PCA to extract the components features and see how much the variance is retained by selection different number of components. We will first isolate the `winning col` we defined previously since those are categorical variables.

We will use 99.5% retained variance as our threshold.

In [71]:
# import packages
from sklearn.decomposition import PCA
pca = PCA(n_components=0.995)

In [278]:
# deal with tennis_all_feature_clean_X_train, tennis_all_feature_clean_X_test
pca_train_1 = pca.fit_transform(tennis_all_feature_clean_X_train.drop(winning_col,1))
pca_test_1 = pca.transform(tennis_all_feature_clean_X_test.drop(winning_col,1))


# create column name list
col_name = []
for i in range(1, pca_train_1.shape[1] + 1):
    col_name.append(f'PCA_{i}')
tennis_all_feature_clean_X_train_PCA = pd.DataFrame(pca_train_1, columns=col_name)
tennis_all_feature_clean_X_test_PCA = pd.DataFrame(pca_test_1, columns=col_name)

# attach back winning_col

for i,col in enumerate(winning_col):
    tennis_all_feature_clean_X_train_PCA[col] = tennis_all_feature_clean_X_train[winning_col].values[:, i]
    tennis_all_feature_clean_X_test_PCA[col] = tennis_all_feature_clean_X_test[winning_col].values[:, i]

In [280]:
# deal with tennis_all_feature_clean_X_train_drop_oppo, tennis_all_feature_clean_X_test_drop_oppo
pca_train_2 = pca.fit_transform(tennis_all_feature_clean_drop_oppo_X_train.drop(winning_col,1))
pca_test_2 = pca.transform(tennis_all_feature_clean_drop_oppo_X_test.drop(winning_col,1))


# create column name list
col_name = []
for i in range(1, pca_train_2.shape[1] + 1):
    col_name.append(f'PCA_{i}')
tennis_all_feature_clean_drop_oppo_X_train_PCA = pd.DataFrame(pca_train_2, columns=col_name)
tennis_all_feature_clean_drop_oppo_X_test_PCA = pd.DataFrame(pca_test_2, columns=col_name)

# attach back winning_col

for i,col in enumerate(winning_col):
    
    tennis_all_feature_clean_drop_oppo_X_train_PCA[col] = \
    tennis_all_feature_clean_drop_oppo_X_train[winning_col].values[:, i]
    
    tennis_all_feature_clean_drop_oppo_X_test_PCA[col] = \
    tennis_all_feature_clean_drop_oppo_X_test[winning_col].values[:, i]

In [72]:
# deal with tennis_all_feature_no_pts_clean_X_train, tennis_all_feature_no_pts_clean_X_test
pca_train_3 = pca.fit_transform(tennis_all_feature_no_pts_clean_X_train.drop(winning_col_2,1))
pca_test_3 = pca.transform(tennis_all_feature_no_pts_clean_X_test.drop(winning_col_2,1))


# create column name list
col_name = []
for i in range(1, pca_train_3.shape[1] + 1):
    col_name.append(f'PCA_{i}')
tennis_all_feature_no_pts_clean_X_train_PCA = pd.DataFrame(pca_train_3, columns=col_name)
tennis_all_feature_no_pts_clean_X_test_PCA = pd.DataFrame(pca_test_3, columns=col_name)

# attach back winning_col

for i,col in enumerate(winning_col_2):
    tennis_all_feature_no_pts_clean_X_train_PCA[col] = tennis_all_feature_no_pts_clean_X_train[winning_col_2].values[:, i]
    tennis_all_feature_no_pts_clean_X_test_PCA[col] = tennis_all_feature_no_pts_clean_X_test[winning_col_2].values[:, i]

In [73]:
# deal with tennis_all_feature_no_pts_clean_X_train, tennis_all_feature_no_pts_clean_X_test
pca_train_4 = pca.fit_transform(tennis_all_feature_no_pts_clean_drop_oppo_X_train.drop(winning_col_2,1))
pca_test_4 = pca.transform(tennis_all_feature_no_pts_clean_drop_oppo_X_test.drop(winning_col_2,1))


# create column name list
col_name = []
for i in range(1, pca_train_4.shape[1] + 1):
    col_name.append(f'PCA_{i}')
tennis_all_feature_no_pts_clean_drop_oppo_X_train_PCA = pd.DataFrame(pca_train_4, columns=col_name)
tennis_all_feature_no_pts_clean_drop_oppo_X_test_PCA = pd.DataFrame(pca_test_4, columns=col_name)

# attach back winning_col

for i,col in enumerate(winning_col_2):
    tennis_all_feature_no_pts_clean_drop_oppo_X_train_PCA[col] = tennis_all_feature_no_pts_clean_drop_oppo_X_train[winning_col_2].values[:, i]
    tennis_all_feature_no_pts_clean_drop_oppo_X_test_PCA[col] = tennis_all_feature_no_pts_clean_drop_oppo_X_test[winning_col_2].values[:, i]

So far we have done many works on feature engineering, feature selection, dimension reduction and we are ready to export all the data we have processed for our model.

In [323]:
# exporting data with matches ranging from 2009-2017, included ranking_points features
tennis_all_feature_clean_X_train.to_csv('../Processed_Data/tennis_all_feature_clean_X_train.csv', index=False)
tennis_all_feature_clean_X_test.to_csv('../Processed_Data/tennis_all_feature_clean_X_test.csv', index=False)
tennis_all_feature_clean_y_train.to_csv('../Processed_Data/tennis_all_feature_clean_y_train.csv', index=False)
tennis_all_feature_clean_y_test.to_csv('../Processed_Data/tennis_all_feature_clean_y_test.csv', index=False)
tennis_all_feature_clean_drop_oppo_X_train.to_csv('../Processed_Data/tennis_all_feature_clean_drop_oppo_X_train.csv', index=False)
tennis_all_feature_clean_drop_oppo_X_test.to_csv('../Processed_Data/tennis_all_feature_clean_drop_oppo_X_test.csv', index=False)
tennis_all_feature_clean_drop_oppo_y_train.to_csv('../Processed_Data/tennis_all_feature_clean_drop_oppo_y_train.csv', index=False)
tennis_all_feature_clean_drop_oppo_y_test.to_csv('../Processed_Data/tennis_all_feature_clean_drop_oppo_y_test.csv', index=False)

tennis_all_feature_clean_X_train_p_value.to_csv('../Processed_Data/tennis_all_feature_clean_X_train_p_value.csv' \
                                                , index=False)
tennis_all_feature_clean_X_test_p_value.to_csv('../Processed_Data/tennis_all_feature_clean_X_test_p_value.csv' \
                                                , index=False)
tennis_all_feature_clean_drop_oppo_X_train_p_value.to_csv('../Processed_Data/tennis_all_feature_clean_drop_oppo_X_train_p_value.csv' \
                                                          , index=False)
tennis_all_feature_clean_drop_oppo_X_test_p_value.to_csv('../Processed_Data/tennis_all_feature_clean_drop_oppo_X_test_p_value.csv' \
                                                          , index=False)

tennis_all_feature_clean_X_train_tree.to_csv('../Processed_Data/tennis_all_feature_clean_X_train_tree.csv' \
                                                , index=False)
tennis_all_feature_clean_X_test_tree.to_csv('../Processed_Data/tennis_all_feature_clean_X_test_tree.csv' \
                                                , index=False)
tennis_all_feature_clean_drop_oppo_X_train_tree.to_csv('../Processed_Data/tennis_all_feature_clean_drop_oppo_X_train_tree.csv' \
                                                          , index=False)
tennis_all_feature_clean_drop_oppo_X_test_tree.to_csv('../Processed_Data/tennis_all_feature_clean_drop_oppo_X_test_tree.csv' \
                                                          , index=False)

tennis_all_feature_clean_X_train_PCA.to_csv('../Processed_Data/tennis_all_feature_clean_X_train_PCA.csv' \
                                                          , index=False)
tennis_all_feature_clean_X_test_PCA.to_csv('../Processed_Data/tennis_all_feature_clean_X_test_PCA.csv' \
                                                          , index=False)
tennis_all_feature_clean_drop_oppo_X_train_PCA.to_csv('../Processed_Data/tennis_all_feature_clean_drop_oppo_X_train_PCA.csv' \
                                                          , index=False)
tennis_all_feature_clean_drop_oppo_X_test_PCA.to_csv('../Processed_Data/tennis_all_feature_clean_drop_oppo_X_test_PCA.csv' \
                                                          , index=False)

In [74]:
# exporting data with matches ranging from 1991-2017, not included ranking_points features
tennis_all_feature_no_pts_clean_X_train.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_X_train.csv', index=False)
tennis_all_feature_no_pts_clean_X_test.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_X_test.csv', index=False)
tennis_all_feature_no_pts_clean_y_train.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_y_train.csv', index=False)
tennis_all_feature_no_pts_clean_y_test.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_y_test.csv', index=False)
tennis_all_feature_no_pts_clean_drop_oppo_X_train.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_drop_oppo_X_train.csv', index=False)
tennis_all_feature_no_pts_clean_drop_oppo_X_test.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_drop_oppo_X_test.csv', index=False)
tennis_all_feature_no_pts_clean_drop_oppo_y_train.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_drop_oppo_y_train.csv', index=False)
tennis_all_feature_no_pts_clean_drop_oppo_y_test.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_drop_oppo_y_test.csv', index=False)

tennis_all_feature_no_pts_clean_X_train_p_value.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_X_train_p_value.csv' \
                                                , index=False)
tennis_all_feature_no_pts_clean_X_test_p_value.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_X_test_p_value.csv' \
                                                , index=False)
tennis_all_feature_no_pts_clean_drop_oppo_X_train_p_value.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_drop_oppo_X_train_p_value.csv' \
                                                          , index=False)
tennis_all_feature_no_pts_clean_drop_oppo_X_test_p_value.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_drop_oppo_X_test_p_value.csv' \
                                                          , index=False)

tennis_all_feature_no_pts_clean_X_train_tree.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_X_train_tree.csv' \
                                                , index=False)
tennis_all_feature_no_pts_clean_X_test_tree.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_X_test_tree.csv' \
                                                , index=False)
tennis_all_feature_no_pts_clean_drop_oppo_X_train_tree.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_drop_oppo_X_train_tree.csv' \
                                                          , index=False)
tennis_all_feature_no_pts_clean_drop_oppo_X_test_tree.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_drop_oppo_X_test_tree.csv' \
                                                          , index=False)

tennis_all_feature_no_pts_clean_X_train_PCA.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_X_train_PCA.csv' \
                                                          , index=False)
tennis_all_feature_no_pts_clean_X_test_PCA.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_X_test_PCA.csv' \
                                                          , index=False)
tennis_all_feature_no_pts_clean_drop_oppo_X_train_PCA.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_drop_oppo_X_train_PCA.csv' \
                                                          , index=False)
tennis_all_feature_no_pts_clean_drop_oppo_X_test_PCA.to_csv('../Processed_Data/tennis_all_feature_no_pts_clean_drop_oppo_X_test_PCA.csv' \
                                                          , index=False)